## Muhammad Ahsan Athallah
### 1301174068

In [60]:
import pandas as pd
import numpy as np
import math
from numba import jit,cuda
import pickle

## Data Processing

In [35]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')
print(movies.shape)
print(ratings.shape)
print(tags.shape)

(9742, 3)
(100836, 4)
(3683, 4)


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
users = set(ratings['userId'])
movieIds = set(movies['movieId'])
print(len(users))
print(len(movieIds))

610
9742


In [12]:
def mappingRating(colrow,rat):
    ratmap = {}
    for i in range(len(colrow)):
        d = colrow.iloc[i]
        if d['userId'] not in ratmap.keys():
            ratmap[d['userId']] = {}
        if d['movieId'] not in ratmap[d['userId']].keys():
            ratmap[d['userId']][d['movieId']] = rat.iloc[i]
        else:
            print('x')
    return ratmap # format : userId1 { movieId1, movieId2}

In [13]:
def isiMatriks(maprat):
    matRat = mat.copy()
    matProb = mat.copy()
    for user in users:
        for movie in movies['movieId']:
            if user in maprat.keys():
                if movie in maprat[user]:
                    rating = maprat[user][movie]
                    matRat[movie][user] = rating
                    matProb[movie][user] = 1
    return matRat, matProb

##### Buat matriks "cetakan"

In [8]:
tempmat = [0]*len(movieIds)
mat = [tempmat]*len(users)
mat = pd.DataFrame(data=mat, columns=movieIds, index=users)
mat.shape #matriks rating template

(610, 9742)

##### Buat Matiks MovieLens All Ratings

In [14]:
X = ratings.drop(['rating','timestamp'],axis=1)
y = ratings['rating']
map_all = mappingRating(X,y)
mat_rat_all, mat_prob_all = isiMatriks(map_all)
mat_rat_all.to_csv('rating_all.csv',index=False)
mat_prob_all.to_csv('prob_all.csv',index=False)
X.shape

(100836, 2)

#### Harusnya film yang nanti direkomndasikan telah dirating 5

In [162]:
rankTrue = np.array([ np.array([int(j) for j in mat_rat_all.loc[i][mat_rat_all.loc[i] == 5].index.values]) for i in mat_rat_all.index ])
rankTrue[0] #film film yang telah di rating 5 oleh user pertama

array([  47,   50,  101,  151,  157,  163,  216,  231,  260,  333,  362,
        457,  527,  553,  596,  608,  661,  919,  923,  940,  954, 1023,
       1024, 1025, 1029, 1031, 1032, 1049, 1073, 1080, 1089, 1092, 1097,
       1136, 1196, 1197, 1198, 1206, 1210, 1213, 1220, 1222, 1224, 1226,
       1240, 1256, 1270, 1275, 1278, 1282, 1291, 1298, 1517, 1573, 1587,
       1617, 1625, 1732, 1804, 1927, 1954, 2005, 2018, 2033, 2048, 2058,
       2078, 2090, 2094, 2115, 2116, 2137, 2139, 2141, 2161, 2291, 2329,
       2353, 2387, 2395, 2427, 2459, 2470, 2502, 2529, 2542, 2571, 2580,
       2596, 2641, 2654, 2692, 2700, 2716, 2761, 2858, 2872, 2899, 2944,
       2947, 2948, 2949, 2959, 2987, 2991, 2993, 3033, 3034, 3052, 3053,
       3147, 3253, 3273, 3386, 3441, 3448, 3450, 3578, 3671, 3702, 3703,
       3729, 3793, 5060])

In [61]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
list_mat_rat_train = []
list_mat_prob_train = []
test_indexs = []
f = 0
for train_index, test_index in kf.split(ratings):
    f += 1
    print('start fold',f)
    X_train = ratings[ratings.index.isin(train_index)].drop(['rating','timestamp'],axis=1)
    y_train = ratings[ratings.index.isin(train_index)]['rating']
    print('mapping..')
    map_train = mappingRating(X_train,y_train)
    print('filling matrix..')
    mat_rat_train, mat_prob_train = isiMatriks(map_train)
    list_mat_rat_train.append(mat_rat_train)
    list_mat_prob_train.append(mat_prob_train)
    test_indexs.append(test_index)
with open('list_mat_prob_train.pickle', 'wb') as f:
    pickle.dump(list_mat_prob_train, f)
with open('list_mat_rat_train.pickle', 'wb') as f:
    pickle.dump(list_mat_rat_train, f)    
np.save('test_indexs.npy',np.array(test_indexs))
print('saved')

start fold 1
mapping..
filling matrix..
start fold 2
mapping..
filling matrix..
start fold 3
mapping..
filling matrix..
start fold 4
mapping..
filling matrix..
start fold 5
mapping..
filling matrix..


### Preprocess Movies

#### Add given tags

In [36]:
tag = [""]*len(movies)
for i,v in tags.iterrows():
    target = movies.loc[movies['movieId'] == v.movieId]
    if len(target) > 0 :
        tag[target.index[0]] += v.tag.lower() + ' '
movies['tags'] = tag
movies.head()

,movieId,title,genres,tags
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake


#### Extract Genres

In [37]:
#get all genres then store to temporary dataframe
genres = []
for i in range(len(movies)):
    genres +=  movies.iloc[i]['genres'].split('|')
    genres = list(set(genres))
print('number of genres =',len(genres))
trow = [0]*len(genres)
tmat = [trow]*len(movies)
dftemp = pd.DataFrame(data=tmat,columns=genres)
print(dftemp.shape)

#concat the temp df with movies
for i in range(len(movies)):
    genres = movies.iloc[i]['genres'].split('|')
    for g in genres:
        dftemp.iloc[i][g] = 1
movies = pd.concat([movies,dftemp],axis=1)
movies.head()

number of genres = 20
(9742, 20)


,movieId,title,genres,tags,Action,Crime,Mystery,Drama,Thriller,Horror,...,War,Animation,Documentary,Adventure,Fantasy,Comedy,Film-Noir,(no genres listed),Romance,Musical
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun,0,0,0,0,0,0,...,0,1,0,1,1,1,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy old,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


#### Extract Titles

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(movies.title)
print(len(vectorizer.get_feature_names()))
dftemp = pd.DataFrame(data = X.toarray(), columns=list(vectorizer.get_feature_names()))
print(dftemp.shape)
movies = pd.concat([movies,dftemp],axis=1)
movies.head()

9269
(9742, 9269)


,movieId,title,genres,tags,Action,Crime,Mystery,Drama,Thriller,Horror,...,écume,époque,étage,étrange,été,être,ôdishon,ônibus,ôritsu,über
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy old,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Extract Tag

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(movies.tags)
print(len(vectorizer.get_feature_names()))
dftemp = pd.DataFrame(data = X.toarray(), columns=list(vectorizer.get_feature_names()))
print(dftemp.shape)
movies = pd.concat([movies,dftemp],axis=1)
movies.head()

1744
(9742, 1744)


,movieId,title,genres,tags,Action,Crime,Mystery,Drama,Thriller,Horror,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar pixar fun,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy magic board game robin williams game,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy old,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculate cosine similiarity between movies

In [40]:
M = movies.drop(['movieId','title','genres','tags'],axis=1).values
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(M)
sim = pd.DataFrame(data=sim,columns=list(movieIds),index=list(movieIds))
sim.head()

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.544331,0.251976,0.251976,0.222222,0.149071,0.298142,0.408248,0.166667,0.298142,...,0.222222,0.089087,0.149071,0.117851,0.000000,0.301511,0.301511,0.000000,0.117851,0.111111
2,0.544331,1.000000,0.154303,0.154303,0.136083,0.182574,0.182574,0.500000,0.204124,0.365148,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.123091,0.123091,0.000000,0.000000,0.000000
3,0.251976,0.154303,1.000000,0.428571,0.251976,0.169031,0.507093,0.154303,0.188982,0.169031,...,0.125988,0.000000,0.169031,0.000000,0.000000,0.113961,0.113961,0.000000,0.000000,0.125988
4,0.251976,0.154303,0.428571,1.000000,0.251976,0.169031,0.507093,0.154303,0.188982,0.169031,...,0.125988,0.101015,0.338062,0.000000,0.000000,0.113961,0.113961,0.218218,0.000000,0.125988
5,0.222222,0.136083,0.251976,0.251976,1.000000,0.149071,0.399929,0.136083,0.166667,0.149071,...,0.222222,0.178174,0.149071,0.117851,0.117851,0.301511,0.100504,0.000000,0.000000,0.111111


#### Lihat film apa yang mirip dengan Toy Story

In [44]:
for i,v in sim.iloc[0].sort_values(ascending=False)[:6].iteritems():
    sel = movies[movies['movieId'] == i ]
    print(sel.title.values[0],'===', sel.genres.values[0],'===',sel.tags.values[0],'===', v)

Toy Story (1995) === Adventure|Animation|Children|Comedy|Fantasy === pixar pixar fun  === 0.9999999999999999
Toy Story 2 (1999) === Adventure|Animation|Children|Comedy|Fantasy === animation disney funny original pixar sequel tom hanks pixar  === 0.8419185910374267
Toy Story 3 (2010) === Adventure|Animation|Children|Comedy|Fantasy|IMAX ===  === 0.7777777777777777
Antz (1998) === Adventure|Animation|Children|Comedy|Fantasy ===  === 0.6299407883487119
Moana (2016) === Adventure|Animation|Children|Comedy|Fantasy ===  === 0.6299407883487119
Turbo (2013) === Adventure|Animation|Children|Comedy|Fantasy ===  === 0.6299407883487119


## Content-Based Filtering

In [124]:
def learnUserPref(probTrain,f,cosim):    
    print('learning user pref fold',f)
    userPref = []
    for u in probTrain.index:
        pos = list(probTrain.loc[u].loc[probTrain.loc[u] == 1].index)#ambil movieId yang pernah dirating
        if len(pos) > 0: #kalau user pernah nge rating sebuah film
            posSim = sim.loc[pos] # ambil cosim dari movie yang pernah dirating
            mostSim = posSim.max() #untuk setiap movieId, ambil maximum similiarity terhadap movie yang pernah di rating.
            userPref.append(mostSim) #simpan prefrensi seorang user u ke  userPref
        else: 
            simD = [1.0]*len(sim.columns) #buat default semua, user pref ga ada, bergantung pada hasil collaborative filtering
            simD = pd.Series(simD, index=list(movieIds))
            userPref.append(simD)
        if u % 10 == 0:
            uP = pd.DataFrame(userPref, index=probTrain.index[:u])
            uP.to_csv('userPrefFold'+str(f)+'.csv')
            print(u,'saved','userPrefFold'+str(f)+'.csv')
    uP = pd.DataFrame(userPref, index=probTrain.index[:u])
    return uP

In [125]:
# with open('list_mat_prob_train.pickle') as f:
#     list_mat_prob_train = pickle.load(f)
userPrefs = []
for i in range(len(list_mat_prob_train)):
    userPref = learnUserPref(list_mat_prob_train[i],i+1,sim)
    userPrefs.append(userPref)
    with open('userPrefs.pickle', 'wb') as f:
        pickle.dump(userPrefs, f) 

learning user pref fold 1
10 saved userPrefFold1.csv
20 saved userPrefFold1.csv
30 saved userPrefFold1.csv
40 saved userPrefFold1.csv
50 saved userPrefFold1.csv
60 saved userPrefFold1.csv
70 saved userPrefFold1.csv
80 saved userPrefFold1.csv
90 saved userPrefFold1.csv
100 saved userPrefFold1.csv
110 saved userPrefFold1.csv
120 saved userPrefFold1.csv
130 saved userPrefFold1.csv
140 saved userPrefFold1.csv
150 saved userPrefFold1.csv
160 saved userPrefFold1.csv
170 saved userPrefFold1.csv
180 saved userPrefFold1.csv
190 saved userPrefFold1.csv
200 saved userPrefFold1.csv
210 saved userPrefFold1.csv
220 saved userPrefFold1.csv
230 saved userPrefFold1.csv
240 saved userPrefFold1.csv
250 saved userPrefFold1.csv
260 saved userPrefFold1.csv
270 saved userPrefFold1.csv
280 saved userPrefFold1.csv
290 saved userPrefFold1.csv
300 saved userPrefFold1.csv
310 saved userPrefFold1.csv
320 saved userPrefFold1.csv
330 saved userPrefFold1.csv
340 saved userPrefFold1.csv
350 saved userPrefFold1.csv
360

## Collaborative Filtering

In [5]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=100, random_state=42) #LDA approach

In [28]:
def evaluateSVD(res,test):
    sse = []
    for i,v in test.iterrows():
        sse.append(pow(v.rating-res.loc[v.userId][v.movieId],2))
    mse = sum(sse)/len(sse)
    return mse, np.sqrt(mse)

In [12]:
with open('list_mat_rat_train.pickle','rb') as f:
    list_mat_rat_train = pickle.load(f)
test_indexs = np.load('test_indexs.npy',allow_pickle=True)
Rpreds = []

In [29]:
for i in range(len(list_mat_rat_train)):
    print('fold',i+1)
    print('training..')
    result = svd.fit_transform(list_mat_rat_train[i])
    result = np.dot(result,svd.components_)
    result = pd.DataFrame(result,index=list(users),columns=list(movieIds))
    test_fold = ratings.loc[test_indexs[i]]
    print('evaluating..')
    MSE,RMSE = evaluateSVD(result,test_fold)
    print('fold',i+1,'MSE =',MSE, 'RMSE =',RMSE)
    Rpreds.append(result)
    with open('Rpreds.pickle', 'wb') as f:
        pickle.dump(Rpreds, f)   
    print('saved')

fold 1
training..
evaluating..
fold 1 MSE = 14.026476260933185 RMSE = 3.745193754792025
saved
fold 2
training..
evaluating..
fold 2 MSE = 13.541222101353231 RMSE = 3.6798399559428168
saved
fold 3
training..
evaluating..
fold 3 MSE = 13.053029616509153 RMSE = 3.612897675897998
saved
fold 4
training..
evaluating..
fold 4 MSE = 13.18615431800806 RMSE = 3.631274475718967
saved
fold 5
training..
evaluating..
fold 5 MSE = 12.868936806796878 RMSE = 3.5873300387331075
saved


### Get Recommendations -> Prediction will rate * Predicted Rating

In [126]:
with open('Rpreds.pickle','rb') as f:
    Rpreds = pickle.load(f)
with open('userPrefs.pickle','rb') as f:
    userPrefs = pickle.load(f)

In [129]:
recoms_all_fold = []
for fold in range(len(Rpreds)):
    newPred = Rpreds[fold].values * userPrefs[fold].values #ini bagian paralelization
    newPred = pd.DataFrame(newPred, index=range(1,611), columns=movieIds)
    recoms = [list(newPred.loc[i].sort_values(ascending=False)[:20].index) for i in newPred.index]
    recoms = pd.DataFrame(recoms, index=range(1,611), columns=range(1,21))
    recoms_all_fold.append(recoms)

#### Lihat movie apa aja yang direkomandasiin untuk user 1 di Fold 1

In [148]:
movies[movies['movieId'].isin(recoms_all_fold[0].loc[1])].title.values

array(['Grease (1978)', 'Truman Show, The (1998)', 'Titanic (1997)',
       'A.I. Artificial Intelligence (2001)',
       "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
       "Ocean's Eleven (2001)", 'Lilo & Stitch (2002)',
       'Harry Potter and the Chamber of Secrets (2002)',
       'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
       "Monty Python's The Meaning of Life (1983)",
       'Kill Bill: Vol. 1 (2003)', 'The Butterfly Effect (2004)',
       '50 First Dates (2004)', 'Kill Bill: Vol. 2 (2004)',
       'Mean Girls (2004)', 'Meet the Fockers (2004)',
       'X-Men: The Last Stand (2006)', 'Ratatouille (2007)',
       'Guardians of the Galaxy (2014)', 'Zootopia (2016)'], dtype=object)

#### Lihat movie apa aja yang dirating 5 oleh user 1

In [188]:
movies.loc[mat_rat_all.loc[1][mat_rat_all.loc[1] == 5].index].title.values

array(['Mighty Aphrodite (1995)', 'Georgia (1995)',
       'Anne Frank Remembered (1995)', 'Mad Love (1995)',
       'Nine Months (1995)', 'Showgirls (1995)', 'I.Q. (1994)',
       'My Crazy Life (Mi vida loca) (1993)', 'Quiz Show (1994)',
       'River Wild, The (1994)', 'Being Human (1993)',
       'Romper Stomper (1992)', 'Aristocats, The (1970)',
       'Kids in the Hall: Brain Candy (1996)',
       'Ghost in the Shell (Kôkaku kidôtai) (1995)', 'Heavy (1995)',
       'Bound (1996)', 'Killer, The (Die xue shuang xiong) (1989)',
       'Full Metal Jacket (1987)', 'Dead Alive (Braindead) (1992)',
       'Bad Taste (1987)', 'Blob, The (1958)', 'Blood Beach (1981)',
       'Body Parts (1991)', 'Burnt Offerings (1976)', 'Cape Fear (1991)',
       'Cape Fear (1962)', "Preacher's Wife, The (1996)",
       'Jerry Maguire (1996)', 'Ghosts of Mississippi (1996)',
       'Mother (1996)', 'Portrait of a Lady, The (1996)',
       'Zeus and Roxanne (1997)', 'Selena (1997)',
       'Picture Perfec

### Evaluation: True Positif jika Film yang masuk top 20 rekomendasi pernah dirating 5
#### Evaluation method: Recall

In [187]:
for fold in range(len(recoms_all_fold)):
    recall = []
    for i in recoms_all_fold[fold].index:
        tp = 0
        for ratU in recoms_all_fold[fold].loc[i]:
            if ratU in rankTrue[i-1]:
                tp += 1
        recall.append(tp/len(recoms_all_fold[fold].loc[i])) #hitung recall rekomendasi seorang user
    print('recall fold',fold+1,np.mean(recall)) #hitung rata rata recall semua user

recall fold 1 0.22573770491803277
recall fold 2 0.21852459016393444
recall fold 3 0.22844262295081966
recall fold 4 0.2174590163934426
recall fold 5 0.2262295081967213
